In [3]:
import sys
import os
#sys.path.insert(0, "/usr/src/app/app")
def import_path(fullpath):
    """ 
    Import a file with full path specification. Allows one to
    import from anywhere, something __import__ does not do. 
    """
    path, filename = os.path.split(fullpath)
    filename, ext = os.path.splitext(filename)
    sys.path.append(path)
    module = __import__(filename)
   # reload(module) # Might be out of date
    del sys.path[-1]
    return module

import_path("/usr/src/app/app")

<module 'app' from '/usr/src/app/app/__init__.py'>

In [4]:
from app.pkg.ml.try_on.preprocessing.aggregator import ClothProcessor
cp = ClothProcessor()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from PIL import Image
image = Image.open("/usr/src/app/data/etc/1.png")
im_no_back = cp.model_background(image)
im_no_back.save("/usr/src/app/data/etc/1__.png")

In [3]:
import os
from app.pkg.ml.auto_clothing_set.autoset import LocalRecSys

from app.pkg.ml.buffer_converters import BytesConverter
from PIL import Image
from app.pkg.models.app.image_category import ImageCategory
import random
bc = BytesConverter()


DATA_PATH = "/usr/src/app/data"
def read_clothes(sub_folder):
    
    images_folder = os.path.join(DATA_PATH, sub_folder)
    assert os.path.exists(images_folder)
    im_paths = os.listdir(images_folder)
    abs_im_path = [os.path.join(images_folder, im_path) for im_path in im_paths]
    return abs_im_path


outerwear_clothes_paths = read_clothes("outerwear") 
upper_clothes_paths = read_clothes("upper") 
lower_clothes_paths = read_clothes("lower") 
dress_clothes_paths = read_clothes("dresses")

user_images = [f"{DATA_PATH}/human/brayan_krenston.png"]

def convert_cloth_to_list(x, category=None):
    res = []
    for i in x:
        image = Image.open(i)
        im_no_back = cp.model_background(image)
        im_bytes = bc.image_to_bytes(im_no_back)
        res.append(image)
    return res

def convert_user_to_dict(x):
    res = []
    for i in x:
        image = Image.open(i)
        im_no_back = cp.model_background(image)
        im_bytes = bc.image_to_bytes(im_no_back)
        res.append({'image':im_bytes,})
    return res

#        im_white_back = cp.model_background.replace_background_RGBA(im_no_back)


upper_clothes = convert_cloth_to_list(upper_clothes_paths, ImageCategory.UPPER_BODY)
lower_clothes = convert_cloth_to_list(lower_clothes_paths, ImageCategory.LOWER_BODY)
dress_clothes = convert_cloth_to_list(dress_clothes_paths, ImageCategory.DRESSES)
outerwear_clothes = convert_cloth_to_list(outerwear_clothes_paths, ImageCategory.UPPER_BODY)
user_images = convert_user_to_dict(user_images)


/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
from PIL import Image
import requests
from transformers import SamModel, SamProcessor
from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation

# model = SamModel.from_pretrained("nielsr/slimsam-77-uniform").to("cuda:0")
# processor = SamProcessor.from_pretrained("nielsr/slimsam-77-uniform")

processor = SegformerImageProcessor.from_pretrained("sayeed99/segformer_b3_clothes")
model = AutoModelForSemanticSegmentation.from_pretrained("sayeed99/segformer_b3_clothes").to("cuda:0")


In [5]:
import numpy as np
import torch
from torch import nn
def change_contrast(img, level):
    factor = (259 * (level + 255)) / (255 * (259 - level))
    def contrast(c):
        return 128 + factor * (c - 128)
    return img.point(contrast)

def get_im_with_mask(pil_im:Image.Image):
        # new_contr = change_contrast(pil_im.convert('RGB'), 50)
        image = np.array(pil_im.convert('RGB'))

#        print(image.shape)
        # convert('RGB') is for images with h,w,4 shape
        

        inputs = processor(image, return_tensors="pt").to("cuda")
        with torch.no_grad(): 
            outputs = model(**inputs)
            logits = outputs.logits.cpu()

            upsampled_logits = nn.functional.interpolate(
                logits,
                size=pil_im.size[::-1],
                mode="bilinear",
                align_corners=False,
            )

            pred_seg = upsampled_logits.argmax(dim=1)[0]

            cloth_mask = pred_seg!=0
            #masks = processor.image_processor.post_process_masks(outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu())
            
            del inputs
            del outputs

        mask = cloth_mask

        pil_im = Image.fromarray(mask.numpy())

        # if background_color:
        #     no_bg_image = Image.new("RGB", pil_im.size, background_color)
            
        
        no_bg_image = Image.new("RGBA", pil_im.size, (0, 0 ,0 ,0))

        orig_image = Image.fromarray(image[:,:,:])

        no_bg_image.paste(orig_image, mask=pil_im)
        result = no_bg_image
        return result


In [ ]:
# uppers_no_back = [get_im_with_mask(im) for im in upper_clothes ]
# lowers_no_back = [get_im_with_mask(im) for im in lower_clothes ]
# outerwear_no_back = [get_im_with_mask(im) for im in outerwear_clothes ]
# dresses_no_back = [get_im_with_mask(im) for im in dress_clothes ]


In [ ]:
# for i in [*uppers_no_back, *lowers_no_back, *outerwear_no_back, *dresses_no_back]:
#     i.show()